In [19]:
import xarray as xr
from bokeh.plotting import output_file, save, output_notebook
from core_portrait_plot import portrait_plot
import numpy as np

In [20]:
# Enable Bokeh output in the notebook
output_notebook()

Loading BokehJS ...

In [21]:
mdl_list = ['MME', 'CanESM5', 'CMCC-CM2-SR5', 'CNRM-ESM2-1', 'EC-Earth3', 'FGOALS-f3-L', 'HadGEM3-GC31-MM', 'IPSL-CM6A-LR', 'MIROC6', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorCPM1']

filename1 = "/global/cfs/projectdirs/m4581/jungchoi/PMP/GMD2026/Fig9/Fig9.Skill_ACC.TAS.nc"
filename2 = "/global/cfs/projectdirs/m4581/jungchoi/PMP/GMD2026/Fig9/Fig9.Skill_MSSS.TAS.nc"
filename3 = "/global/cfs/projectdirs/m4581/jungchoi/PMP/GMD2026/Fig9/Fig9.Skill_RPC.TAS.nc"

ds1 = xr.open_dataset(filename1)
ds2 = xr.open_dataset(filename2)
ds3 = xr.open_dataset(filename3)

var1_data = ds1["acc"].values
var2_data = ds2["msss"].values
var3_data = ds3["rpc"].values

In [22]:
arctic_index = np.arange(0, 60, 5)
nh_index = np.arange(1, 60, 5)
tropics_index = np.arange(2, 60, 5)
sh_index = np.arange(3, 60, 5)
antarctic_index = np.arange(4, 60, 5)

In [23]:
arctic1_data = []
for a in arctic_index:
    arctic1_data.append(var1_data[a])

nh1_data = []
for nh in nh_index:
    nh1_data.append(var1_data[nh])
    
tropics1_data = []
for t in tropics_index:
    tropics1_data.append(var1_data[t])
    
sh1_data = []
for sh in sh_index:
    sh1_data.append(var1_data[sh])
    
antarctic1_data = []
for an in antarctic_index:
    antarctic1_data.append(var1_data[an])

In [24]:
xaxis_labels = ['LY1', 'LY2', 'LY3', 'LY4', 'LY5', 'LY1-5']
yaxis_labels = mdl_list

In [25]:
p = portrait_plot(
    [arctic1_data, nh1_data, tropics1_data, sh1_data, antarctic1_data][::-1], 
    static=True,
    xaxis_labels=xaxis_labels,
    yaxis_labels=yaxis_labels,
    xaxis_fontsize=13,
    yaxis_fontsize=13,
    height=1000, width=1000,
    cmap="RdBu_r",
    vrange=(-4,4),
    #cmap_bounds=[-4, -3.5, -3, -2.5, -2, -1.5, -1, -.5, 0, .5, 1, 1.5, 2, 2.5, 3, 3.5, 4],
    title="TAS ACC",
    show_plot=False
    )

In [26]:
# Adding some customizations
from bokeh.plotting import figure, show
from bokeh.models import Span, Title, Div
from bokeh.io import output_file, save
from bokeh.layouts import column, row

line_locations = np.arange(0, 12, 1)

for loc in line_locations:
    span = Span(location=loc, dimension='width', line_color='black', line_width=2)
    p.add_layout(span)
    
for loc in line_locations:
    span = Span(location=loc, dimension='height', line_color='black', line_width=0.5)
    p.add_layout(span)
    
legend_title = Title(text="[K]", align="center", text_font_size="13pt")
p.add_layout(legend_title, 'right')
p.title.text_font_size = '14pt'

In [27]:
# custom function that creates an array of img URLs for each glyph, with custom sort
# use xaxis labels and mdl_list
def get_img_urls(mode: str='ACC', var: str='TAS'):
    urls = []
    url_head = f'https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/dcpp/skill_score/Skill_{mode}.{var}'
    for n in range(5):
        for mdl in mdl_list[::-1]:
            for lbl in xaxis_labels:
                url = f'{url_head}.{mdl}.{lbl}.png'
                urls.append(url)
    return urls

In [28]:
acc_img_urls = get_img_urls(mode='ACC')
msss_img_urls = get_img_urls(mode='MSSS')
rpc_img_urls = get_img_urls(mode='RPC')

In [29]:
from matplotlib.colors import ListedColormap

colors1 = [
     '#92c5de', '#d1e5f0',  '#f7f7f7', 
    '#ffffcc', '#ffeda0', '#fed976', '#feb24c', '#fd8d3c', '#fc4e2a', '#e31a1c', '#bd0026', '#800026', 
    '#67000d', '#a50f15'
]
cmap1 = ListedColormap(colors1)
bounds1 = [-.3, -.2, -.1, 0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1.]

In [30]:
# creating interactive version
p = portrait_plot(
    [arctic1_data, nh1_data, tropics1_data, sh1_data, antarctic1_data][::-1],
    img_url = acc_img_urls,
    img_width=650,
    clickable = True,
    xaxis_labels=xaxis_labels,
    yaxis_labels=yaxis_labels,
    xaxis_fontsize=13,
    yaxis_fontsize=13,
    height=1000, width=1000,
    cmap=cmap1,
    #vrange=(-0.3,1),
    cmap_bounds=bounds1,
    title="TAS ACC (s1960-s2016) (ref: ERA5)",
    show_plot=False
    )

line_locations = np.arange(0, 13, 1)

for loc in line_locations:
    h_lines = Span(location=loc, dimension='width', line_color='black', line_width=3)
    p.add_layout(h_lines)
    
for loc in line_locations:
    v_lines = Span(location=loc, dimension='height', line_color='black', line_width=0.7)
    p.add_layout(v_lines)
    
end_line = Span(location=10.99, dimension='height', line_color='black', line_width=0.7)
p.add_layout(end_line)

hist_line = Span(location=10, dimension='height', line_color='black', line_width=3)
p.add_layout(hist_line)
    
legend_title = Title(text="", align="center", text_font_size="13pt")
p.add_layout(legend_title, 'right')
p.title.text_font_size = '14pt'

# Add text to top of the page
text_title = """
<div class=title>
<center>
<h1>CMIP6 DCPP TAS ACC</h1>
</center>
</div>
"""
logos = """
<div class=title>
<center>
<br>

<a href="https://pcmdi.llnl.gov/" target="_blank">
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_PCMDI.png" height="50" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="PCMDI"
    title="PCMDI"></a>

<a href=https://climatemodeling.science.energy.gov/program/regional-global-model-analysis target=_blank>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_doe.png height="45" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="US Sponsor: DOE BER RGMA"
    title="US Sponsor: DOE BER RGMA"></a>

</center>
</div>
<br>
"""

text_top = text_title + logos
text_top += """
<style>
.title {
    width: 160%;
}
.box1 {
    background-color: #E8E8E8;
    width: 160%;
    padding: 10px;
    margin: 0px;
}
.dropdown {
    background-color: white;
    width: 100%;
    padding: 10px;
    margin: 0px;
}
</style>

<div class=box1>
<p><b><font size=2 color=#e67e22>Supported browsers: Chrome, Firefox, Safari, Microsoft Edge</font></b></p>

<p><b><font size=4 color=darkblue> Reference and Contributions </font></b><br>
- Citation: Choi et al. (2026, in prep.)<br>
- The calculation has been conducted and dive down plots were generated by Jung Choi.<br>
- The Interactive Portrait Plot and webpage are generated by <a href=https://people.llnl.gov/chang61 target=_blank>Kristin Chang</a> and <a href=https://pcmdi.llnl.gov/staff/lee/ target=_blank>Jiwoo Lee</a> (PCMDI/LLNL).<br>
"""

overall_text_usage = """
<p><b><font size=4 color=darkblue> Usage</b> <i>(Important for navigating to dive-down diagnostics)</i></font><br>
- Hover your mouse over boxes will show tooltips for metric values and preview of dive down plots.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/tap.svg style="height: 15px;">: Turn on/off <b>Tap</b> icon to enable/disable clicking on boxes.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/save.svg style="height: 15px;">: Click <b>Save</b> icon to download a static HTML file of the plot.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/hover.svg style="height: 15px;">: Turn on/off <b>Hover</b> icon enable/disable showing tooltips.<br>
"""

legend_img = """
<style>
h3 {
  float:right;
  margin: 10px -360px;
}
</style>
<h3>Region Legend</h3>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/dcpp/mean_bias/region_legend.png height="100"
    style="vertical-align:middle;horizontal-align:right;margin:30px -400px;float:right;"
    alt="PCMDI Mean Bias Region Legend"
    title="PCMDI Mean Bias Region Legend">
"""

text_top += overall_text_usage + "</div>" + legend_img

# LLNL Disclaimer
#

text_bottom = """
<style>
.bottom {
    width: 1000px;
}
.fig_desc {
    width:1000px;
    font-size:11pt;
}
table {
    font-family: arial, sans-serif;
    border-collapse: collapse;
    width: 100%;
}
td, th {
    border: 1px solid #dddddd;
    text-align: left;
    padding: 8px;
    text-align: center;
}
tr:nth-child(even) {
    background-color: #dddddd;
}
</style>
"""

text_bottom += """
<div class=fig_desc>
<p><b><font size=4 color=darkblue> Figure description</b></font><br>
This portrait plot shows anomaly correlation coefficient (ACC) of TAS for the multi-model ensemble mean (MME) and each model at LY1, LY2, LY3, LY4, LY5, and LY1-5. Skill scores are calculated using the annual-mean TAS at each grid cell, and then averaged across the regions. The interactive subplots show the spatial distribution of TAS ACC. (a) ACC calculated from all available DCPP experiments initialized from 1960 to 2016. (b and c) Same as (a) but for the common evaluation period of 1965-2014 from DCPP and HIST simulations, respectively. Yellow lines in (a, b, and c) indicate an ACC of 0.6. (d) The difference between (b) and (c) represents the added skill due to initialization. Red regions in (d) indicate areas where initialization improves prediction skill. 
</div>
<br><br>
<div class=bottom>
<br><p>
<a href="https://www.llnl.gov" target=_blank>
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_llnl-600.png" 
alt="LLNL" style="float:left;height:80px;margin-right:10px">
</a>
The work of PCMDI and results presented on this web page are supported by the <a href=https://eesm.science.energy.gov/program-area/regional-global-model-analysis>Regional and Global Model Analysis (RGMA) program area
</a> under the <a href=https://eesm.science.energy.gov/about>Earth and Environmental System Modeling (EESM) program
</a> within the <a href=https://science.osti.gov/ber/Research/eessd>Earth and Environmental Systems Sciences Division (EESSD)
</a> of the <a href=https://science.osti.gov/>United States Department of Energy's (DOE) Office of Science (OSTI)
</a>. The work of PCMDI is performed under the auspices of the U.S. DOE by <a href=https://pls.llnl.gov/>Lawrence Livermore National Laboratory (LLNL)
</a> under contract DE-AC52-07NA27344. LLNL-WEB-812310.</p>
</div>
"""

# combine page
page = column(Div(text=text_top),
              p,
              Div(text=text_bottom)) 
    
#show(p)
output_file(filename="interactive_TAS_skill_ACC_portrait_plot.html", title="Interactive TAS ACC Portrait Plot")
save(page, filename='./interactive_TAS_skill_ACC_portrait_plot.html')

'/global/cfs/cdirs/m4581/jungchoi/PMP/interactive_Mean_bias_portrait/interactive_TAS_skill_ACC_portrait_plot.html'

In [31]:
# Repeat process for 2nd var
arctic2_data = []
for a in arctic_index:
    arctic2_data.append(var2_data[a])

nh2_data = []
for nh in nh_index:
    nh2_data.append(var2_data[nh])
    
tropics2_data = []
for t in tropics_index:
    tropics2_data.append(var2_data[t])
    
sh2_data = []
for sh in sh_index:
    sh2_data.append(var2_data[sh])
    
antarctic2_data = []
for an in antarctic_index:
    antarctic2_data.append(var2_data[an])

In [32]:
# creating interactive version
p = portrait_plot(
    [arctic2_data, nh2_data, tropics2_data, sh2_data, antarctic2_data][::-1],
    img_url = msss_img_urls,
    img_width = 650,
    clickable = True,
    xaxis_labels=xaxis_labels,
    yaxis_labels=yaxis_labels,
    xaxis_fontsize=13,
    yaxis_fontsize=13,
    height=1000, width=1000,
    cmap=cmap1,
    #vrange=(-0.3,1),
    cmap_bounds=bounds1,
    title="TAS MSSS (s1960-s2016) (ref: ERA5)",
    show_plot=False
    )

line_locations = np.arange(0, 13, 1)

for loc in line_locations:
    h_lines = Span(location=loc, dimension='width', line_color='black', line_width=3)
    p.add_layout(h_lines)
    
for loc in line_locations:
    v_lines = Span(location=loc, dimension='height', line_color='black', line_width=0.7)
    p.add_layout(v_lines)
    
end_line = Span(location=10.99, dimension='height', line_color='black', line_width=0.7)
p.add_layout(end_line)

hist_line = Span(location=10, dimension='height', line_color='black', line_width=3)
p.add_layout(hist_line)
    
legend_title = Title(text="", align="center", text_font_size="13pt")
p.add_layout(legend_title, 'right')
p.title.text_font_size = '14pt'

# Add text to top of the page
text_title = """
<div class=title>
<center>
<h1>CMIP6 DCPP TAS MSSS</h1>
</center>
</div>
"""
logos = """
<div class=title>
<center>
<br>

<a href="https://pcmdi.llnl.gov/" target="_blank">
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_PCMDI.png" height="50" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="PCMDI"
    title="PCMDI"></a>

<a href=https://climatemodeling.science.energy.gov/program/regional-global-model-analysis target=_blank>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_doe.png height="45" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="US Sponsor: DOE BER RGMA"
    title="US Sponsor: DOE BER RGMA"></a>

</center>
</div>
<br>
"""

text_top = text_title + logos
text_top += """
<style>
.title {
    width: 160%;
}
.box1 {
    background-color: #E8E8E8;
    width: 160%;
    padding: 10px;
    margin: 0px;
}
.dropdown {
    background-color: white;
    width: 100%;
    padding: 10px;
    margin: 0px;
}
</style>

<div class=box1>
<p><b><font size=2 color=#e67e22>Supported browsers: Chrome, Firefox, Safari, Microsoft Edge</font></b></p>

<p><b><font size=4 color=darkblue> Reference and Contributions </font></b><br>
- Citation: Choi et al. (2026, in prep.)<br>
- The calculation has been conducted and dive down plots were generated by Jung Choi.<br>
- The Interactive Portrait Plot and webpage are generated by <a href=https://people.llnl.gov/chang61 target=_blank>Kristin Chang</a> and <a href=https://pcmdi.llnl.gov/staff/lee/ target=_blank>Jiwoo Lee</a> (PCMDI/LLNL).<br>
"""

overall_text_usage = """
<p><b><font size=4 color=darkblue> Usage</b> <i>(Important for navigating to dive-down diagnostics)</i></font><br>
- Hover your mouse over boxes will show tooltips for metric values and preview of dive down plots.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/tap.svg style="height: 15px;">: Turn on/off <b>Tap</b> icon to enable/disable clicking on boxes.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/save.svg style="height: 15px;">: Click <b>Save</b> icon to download a static HTML file of the plot.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/hover.svg style="height: 15px;">: Turn on/off <b>Hover</b> icon enable/disable showing tooltips.<br>
"""

legend_img = """
<style>
h3 {
  float:right;
  margin: 10px -360px;
}
</style>
<h3>Region Legend</h3>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/dcpp/mean_bias/region_legend.png height="100"
    style="vertical-align:middle;horizontal-align:right;margin:30px -400px;float:right;"
    alt="PCMDI Mean Bias Region Legend"
    title="PCMDI Mean Bias Region Legend">
"""

text_top += overall_text_usage + "</div>" + legend_img

# LLNL Disclaimer
#

text_bottom = """
<style>
.bottom {
    width: 1000px;
}
.fig_desc {
    width:1000px;
    font-size:11pt;
}
table {
    font-family: arial, sans-serif;
    border-collapse: collapse;
    width: 100%;
}
td, th {
    border: 1px solid #dddddd;
    text-align: left;
    padding: 8px;
    text-align: center;
}
tr:nth-child(even) {
    background-color: #dddddd;
}
</style>
"""

text_bottom += """
<div class=fig_desc>
<p><b><font size=4 color=darkblue> Figure description</b></font><br>
This portrait plot shows mean squared skill score (MSSS) of TAS for the multi-model ensemble mean (MME) and each model at LY1, LY2, LY3, LY4, LY5, and LY1-5. Skill scores are calculated using the annual-mean TAS at each grid cell, and then averaged across the regions. The interactive subplots show the spatial distribution of TAS MSSS. (a) MSSS calculated from all available DCPP experiments initialized from 1960 to 2016. (b and c) Same as (a) but for the common evaluation period of 1965-2014 from DCPP and HIST simulations, respectively. Yellow lines in (a, b, and c) indicate a MSSS of 0.6. (d) The difference between (b) and (c) represents the added skill due to initialization. Red regions in (d) indicate areas where initialization improves prediction skill. </div>
<br><br>
<div class=bottom>
<br><p>
<a href="https://www.llnl.gov" target=_blank>
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_llnl-600.png" 
alt="LLNL" style="float:left;height:80px;margin-right:10px">
</a>
The work of PCMDI and results presented on this web page are supported by the <a href=https://eesm.science.energy.gov/program-area/regional-global-model-analysis>Regional and Global Model Analysis (RGMA) program area
</a> under the <a href=https://eesm.science.energy.gov/about>Earth and Environmental System Modeling (EESM) program
</a> within the <a href=https://science.osti.gov/ber/Research/eessd>Earth and Environmental Systems Sciences Division (EESSD)
</a> of the <a href=https://science.osti.gov/>United States Department of Energy's (DOE) Office of Science (OSTI)
</a>. The work of PCMDI is performed under the auspices of the U.S. DOE by <a href=https://pls.llnl.gov/>Lawrence Livermore National Laboratory (LLNL)
</a> under contract DE-AC52-07NA27344. LLNL-WEB-812310.</p>
</div>
"""

# combine page
page = column(Div(text=text_top),
              p,
              Div(text=text_bottom)) 
    
#show(p)
output_file(filename="interactive_TAS_skill_MSSS_portrait_plot.html", title="Interactive TAS MSSS Portrait Plot")
save(page, filename='./interactive_TAS_skill_MSSS_portrait_plot.html')

'/global/cfs/cdirs/m4581/jungchoi/PMP/interactive_Mean_bias_portrait/interactive_TAS_skill_MSSS_portrait_plot.html'

In [33]:
# Repeat process for 3rd var
arctic3_data = []
for a in arctic_index:
    arctic3_data.append(var3_data[a])

nh3_data = []
for nh in nh_index:
    nh3_data.append(var3_data[nh])
    
tropics3_data = []
for t in tropics_index:
    tropics3_data.append(var3_data[t])
    
sh3_data = []
for sh in sh_index:
    sh3_data.append(var3_data[sh])
    
antarctic3_data = []
for an in antarctic_index:
    antarctic3_data.append(var3_data[an])

In [34]:
from matplotlib.colors import ListedColormap

In [35]:
colors2 = [
   # '#cccccc', '#081d58', '#253494', '#3f007d', '#54278f', '#6a51a3', 
   # '#807dba', '#9e9ac8', '#bcbddc', '#dadaeb', '#efedf5',  '#e08214', '#b35806']
 #'#238443', '#41ab5d', '#78c679', '#addd8e', '#d9f0a3', '#f7fcb9',  '#fde0dd', '#fa9fb5', '#f768a1']
 '#238443', '#41ab5d', '#78c679', '#addd8e', '#d9f0a3', '#f7fcb9',  '#fa9fb5', '#f768a1']
bounds2 = [ .4, .5, .6, .7, .8, .9, 1.0, 1.1, 1.2]
cmap2 = ListedColormap(colors2)

In [36]:
# creating interactive version
p = portrait_plot(
    [arctic3_data, nh3_data, tropics3_data, sh3_data, antarctic3_data][::-1],
    img_url = rpc_img_urls,
    img_width = 650,
    clickable = True,
    xaxis_labels=xaxis_labels,
    yaxis_labels=yaxis_labels,
    xaxis_fontsize=13,
    yaxis_fontsize=13,
    height=1000, width=1000,
    cmap=cmap2,
    #vrange=(-0.3,1),
    cmap_bounds=bounds2,
    custom_vmin = 0.4,
    custom_vmax = 1.2,
    title="TAS RPC (s1960-s2016) (ref: ERA5)",
    show_plot=False
    )

line_locations = np.arange(0, 13, 1)

for loc in line_locations:
    h_lines = Span(location=loc, dimension='width', line_color='black', line_width=3)
    p.add_layout(h_lines)
    
for loc in line_locations:
    v_lines = Span(location=loc, dimension='height', line_color='black', line_width=0.7)
    p.add_layout(v_lines)
    
end_line = Span(location=10.99, dimension='height', line_color='black', line_width=0.7)
p.add_layout(end_line)

hist_line = Span(location=10, dimension='height', line_color='black', line_width=3)
p.add_layout(hist_line)
    
legend_title = Title(text="", align="center", text_font_size="13pt")
p.add_layout(legend_title, 'right')
p.title.text_font_size = '14pt'

# Add text to top of the page
text_title = """
<div class=title>
<center>
<h1>CMIP6 DCPP TAS RPC</h1>
</center>
</div>
"""
logos = """
<div class=title>
<center>
<br>

<a href="https://pcmdi.llnl.gov/" target="_blank">
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_PCMDI.png" height="50" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="PCMDI"
    title="PCMDI"></a>

<a href=https://climatemodeling.science.energy.gov/program/regional-global-model-analysis target=_blank>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_doe.png height="45" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="US Sponsor: DOE BER RGMA"
    title="US Sponsor: DOE BER RGMA"></a>

</center>
</div>
<br>
"""

text_top = text_title + logos
text_top += """
<style>
.title {
    width: 160%;
}
.box1 {
    background-color: #E8E8E8;
    width: 160%;
    padding: 10px;
    margin: 0px;
}
.dropdown {
    background-color: white;
    width: 100%;
    padding: 10px;
    margin: 0px;
}
</style>

<div class=box1>
<p><b><font size=2 color=#e67e22>Supported browsers: Chrome, Firefox, Safari, Microsoft Edge</font></b></p>

<p><b><font size=4 color=darkblue> Reference and Contributions </font></b><br>
- Citation: Choi et al. (2026, in prep.)<br>
- The calculation has been conducted and dive down plots were generated by Jung Choi.<br>
- The Interactive Portrait Plot and webpage are generated by <a href=https://people.llnl.gov/chang61 target=_blank>Kristin Chang</a> and <a href=https://pcmdi.llnl.gov/staff/lee/ target=_blank>Jiwoo Lee</a> (PCMDI/LLNL).<br>
"""

overall_text_usage = """
<p><b><font size=4 color=darkblue> Usage</b> <i>(Important for navigating to dive-down diagnostics)</i></font><br>
- Hover your mouse over boxes will show tooltips for metric values and preview of dive down plots.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/tap.svg style="height: 15px;">: Turn on/off <b>Tap</b> icon to enable/disable clicking on boxes.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/save.svg style="height: 15px;">: Click <b>Save</b> icon to download a static HTML file of the plot.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/hover.svg style="height: 15px;">: Turn on/off <b>Hover</b> icon enable/disable showing tooltips.<br>
"""

legend_img = """
<style>
h3 {
  float:right;
  margin: 10px -360px;
}
</style>
<h3>Region Legend</h3>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/dcpp/mean_bias/region_legend.png height="100"
    style="vertical-align:middle;horizontal-align:right;margin:30px -400px;float:right;"
    alt="PCMDI Mean Bias Region Legend"
    title="PCMDI Mean Bias Region Legend">
"""

text_top += overall_text_usage + "</div>" + legend_img

# LLNL Disclaimer
#

text_bottom = """
<style>
.bottom {
    width: 1000px;
}
.fig_desc {
    width:1000px;
    font-size:11pt;
}
table {
    font-family: arial, sans-serif;
    border-collapse: collapse;
    width: 100%;
}
td, th {
    border: 1px solid #dddddd;
    text-align: left;
    padding: 8px;
    text-align: center;
}
tr:nth-child(even) {
    background-color: #dddddd;
}
</style>
"""

text_bottom += """
<div class=fig_desc>
<p><b><font size=4 color=darkblue> Figure description</b></font><br>
This portrait plot shows ratio of predictable components (RPC) of TAS for the multi-model ensemble mean (MME) and each model at LY1, LY2, LY3, LY4, LY5, and LY1-5. Skill scores are calculated using the annual-mean TAS at each grid cell, and then averaged across the regions. RPC is only calculated in regions where ACC > 0.   The interactive subplots show the spatial distribution of TAS RPC. (a) RPC calculated from all available DCPP experiments initialized from 1960 to 2016. (b and c) Same as (a) but for the common evaluation period of 1965-2014 from DCPP and HIST simulations, respectively. (d) The difference between (b) and (c) represents the added skill due to initialization. Red regions in (d) indicate areas where initialization increases the RPC values. 
</div>
<br><br>
<div class=bottom>
<br><p>
<a href="https://www.llnl.gov" target=_blank>
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_llnl-600.png" 
alt="LLNL" style="float:left;height:80px;margin-right:10px">
</a>
The work of PCMDI and results presented on this web page are supported by the <a href=https://eesm.science.energy.gov/program-area/regional-global-model-analysis>Regional and Global Model Analysis (RGMA) program area
</a> under the <a href=https://eesm.science.energy.gov/about>Earth and Environmental System Modeling (EESM) program
</a> within the <a href=https://science.osti.gov/ber/Research/eessd>Earth and Environmental Systems Sciences Division (EESSD)
</a> of the <a href=https://science.osti.gov/>United States Department of Energy's (DOE) Office of Science (OSTI)
</a>. The work of PCMDI is performed under the auspices of the U.S. DOE by <a href=https://pls.llnl.gov/>Lawrence Livermore National Laboratory (LLNL)
</a> under contract DE-AC52-07NA27344. LLNL-WEB-812310.</p>
</div>
"""

# combine page
page = column(Div(text=text_top),
              p,
              Div(text=text_bottom)) 
    
#show(p)
output_file(filename="interactive_TAS_skill_RPC_portrait_plot.html", title="Interactive TAS RPC Portrait Plot")
save(page, filename='./interactive_TAS_skill_RPC_portrait_plot.html')

'/global/cfs/cdirs/m4581/jungchoi/PMP/interactive_Mean_bias_portrait/interactive_TAS_skill_RPC_portrait_plot.html'